In [ ]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')


# Data

Every time a user opens a mobile app, an auction is going on behind the scenes. The highest bidder gets to advertise his ad to the user.

## Auctions Table


In [ ]:
sql_query = 'SELECT * FROM auctions;'
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

## App Vectors table

We've gathered the first few sentences from the app store description and embedded it with a [model](https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)


In [ ]:
sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        df_of_embeddings = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~df_of_embeddings["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

df_of_embeddings


We can use the `<=>` operator to run vector search within the database


In [ ]:
vec = json.loads(df_of_embeddings.embedding[0]) # get the first embedding
print ("Embedding size: {l}".format(l=len(vec)))

sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
'''
with engine.connect() as db_con:
    similar_embeddings = pd.read_sql(sql_query, con=db_con)

similar_embeddings

# What you need to do

## The hypothesis

We assume that apps with similar desciptions, would have a similar asking price in the auctions (`sentPrice` column).

Use cosine similarity (`<=>`) on the embeddings to find similar apps, and any statistical tools you find suitable to prove or disprove this hypothesis.

## Is it consistent?

There are several other features in the auctions table (such as `CountryCode` and `OS`),
Do your findings hold for those as well?


In [ ]:
# let's install some additional libraries in order to visualize the data and compare different models
!pip install scikit-learn
!pip install xgboost
!pip install seaborn

# import the required libraries
import itertools
import numpy as np 
import xgboost as xgb
import seaborn as sns # for data visualization
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error

from sklearn.metrics.pairwise import cosine_similarity # for computing cosine similarity
from sklearn.model_selection import train_test_split # for splitting the data into training, validation and test sets

# will use us for selecting random apps to put in the validation and test sets
import random


# ------------------- Planning -------------------


# My work plan for testing the hypotesis:

(1) Start with some descriptive statistics in order to better understand the data and what decsisions to make

(2) Evalute the quality of the embeddings, I would assume that apps with similar semantic description would have similar vector embeddings. If it's not the case, maybe we should use different embedding model.

(3) Design an experiment which compares a baseline model which was trained without embeddings and a model which takes into account the apps' embeddings

(4) Execution - Feature engineering, splitting the data in a way which avoids data leakage, training

(5) Results

(6) Testing the contribution of additional features

(7) Conclusions

For that, we can create a confusion matrix which contains the cosine similarity values.


# ------------------- (1) Descriptive statistics -------------------

Let's analyze the sentPrice values with respect to each app.

This will enable us to understand the number of apps present, their sentPrice distribution, and their percentage out of all the data.


In [ ]:
# Here I Group by the df based on the 'bundleId' column and calculate descriptive statistics for 'sentPrice'
grouped_stats = df.groupby('bundleId')['sentPrice'].describe()

# Calculate percentage of each group out of all records
grouped_stats['percentage'] = grouped_stats['count'] / len(df) * 100

print(grouped_stats)

# Notes from the descriptive analysis:

It seems that some apps have less than 5% of the samples.

We should keep some apps for the validation and test data, in order to make sure the embeddings of these apps would not leak in to training data.

More about it later on.


# ------------------- (2) Embeddings qualitative evaluation -------------------

Based on the pervious analysis, it seems that there are only 18 apps.

Let's create a confusion matrix in order to easily examine the similarity between the apps.


In [ ]:
# create a function that computes the cosine similarity between two embeddings
def create_cos_sim_df(df_of_embeddings):
    
    embeddings_dict = {}
    
    # (1) Convert string representations of lists to actual lists
    df_of_embeddings['embedding'] = df_of_embeddings['embedding'].apply(json.loads)

    # store embeddings in a dict:
    for idx, row in df_of_embeddings.iterrows():
        embeddings_dict[row['bundleId']] = row['embedding']
        
    # (2) Initialize a matrix to store cosine similarities
    num_embeddings = len(df_of_embeddings)
    cos_sim_matrix = np.zeros((num_embeddings, num_embeddings)) 

    # (3) Compute cosine similarity between each pair of embeddings
    for i, j in itertools.product(range(num_embeddings), range(num_embeddings)):
        # ignore the diagonal elements
        if i != j: 
            cos_sim_matrix[i, j] = cosine_similarity([df_of_embeddings['embedding'][i]], [df_of_embeddings['embedding'][j]])[0, 0]

    # (4) reate DataFrame for cosine similarity matrix
    cos_sim_df = pd.DataFrame(cos_sim_matrix, columns=df_of_embeddings['bundleId'], index=df_of_embeddings['bundleId'])

    return cos_sim_df,embeddings_dict

cos_sim_df,embeddings_dict = create_cos_sim_df(df_of_embeddings)

## Visualize confusion matrix which reflects the cosine similarity values:


In [ ]:
# Create heatmap of cosine similarity values
plt.figure(figsize=(10, 8))
sns.heatmap(cos_sim_df, annot=True, cmap='coolwarm')
plt.title('Cosine Similarity Heatmap')
plt.xlabel('bundleId')
plt.ylabel('bundleId')
plt.show()

## Assessing Similarity of Apps Based on Cosine Similarity Values

For example, consider the apps "com.volt.dresstoimpress" and "1569586264" with a cosine similarity of 1.

The app description for "com.volt.dresstoimpress" is: "Choose the appropriate outfit to make it through different social events!"

The app description for "1569586264" is: "Choose the appropriate outfit to make it through different social events!"

This analysis suggests that these might be the same apps!

---

## Let's also analyze two embeddings that are not identical but have a high cosine similarity of 0.92.

The app description for "com.loop.match3d" is: "Get ready for a new, challenging and original matching pairs brain game."

The app description for "1502447854" is: "Get ready for a new, challenging and original matching pairs game."

These descriptions are very close!

---

## Lastly, let's examine a couple of apps with medium similarity = 0.5.

The app description for "com.tintash.nailsalon" is:

"It is manicure madness over here and it’s your time to become the greatest Nail Salon of 2021! All you need to do is scrape, clip, paint, polish and perfect your client’s nails and you will be raking in the money in no time! Just don’t mess up! People don’t like when you accidentally pull their nails off. Ouch!"

The app description for "com.volt.dresstoimpress" is:

"Choose the appropriate outfit to make it through different social events!".

Conclusion - not so similar.

# Final conclusion - the embeddings seems to reflect the semantical similarity between apps


# ------------------- (3) Experiment design -------------------

- I would suggest training a baseline model aimed at predicting the sentPrice target variable.
  Input - bids' timestamp.
  Output - sentPrice.

- The baseline model would be compared with a model trained on additional features that reflect the app embeddings.
  Input - bids' timestamp, embeddings (after some standard clustering).
  Output - sentPrice.

- The model which will use me is the popular XGBOOST - this model known for it's powerful predictive and can also provide us insights regarding the feature importance
  Evaluation metric - RMSE.

- If the baseline_RMSE < model_with_embedding_RMSE, then the hypothesis would be confirmed.

# note

- Since there are some apps with low represntation, I would suggest randomly assign some of them to the validation and testing sets.
  This would prevent data leakage when performing the embeddings clustering.


# ------------------- (4) Execution -------------------

This phase includes:

(a) Feature engineering - I will transform the timestamp variable into year,month, day, hour, minute,second and day_of_week.

(b) I will randomly some of the low appearance apps to the validation and testing data

(c) Training a model without the apps' embeddings

(d) Clustering the apps based on cosine similarity + generate one hot encoding feature for each cluster

(e) Train a model with the app's embeddings


# 4a. Split the data into training, validation and test sets


In [ ]:
# Create features from the eventTimestamp variable

def timestamp_to_features(df):
    # Create features from the eventTimestamp variable
    df['eventTimestamp'] = pd.to_numeric(df['eventTimestamp'], errors='coerce')
    df['eventTimestamp'] = pd.to_datetime(df['eventTimestamp'] / 1000, unit='s')

    # Extract year, month, day, hour, minute, second, and day of the week
    df['year'] = df['eventTimestamp'].dt.year
    df['month'] = df['eventTimestamp'].dt.month
    df['day'] = df['eventTimestamp'].dt.day
    df['hour'] = df['eventTimestamp'].dt.hour
    df['minute'] = df['eventTimestamp'].dt.minute
    df['second'] = df['eventTimestamp'].dt.second
    df['day_of_week'] = df['eventTimestamp'].dt.dayofweek 

    # Drop the original eventTimestamp column
    df = df.drop(columns=['eventTimestamp'])
    
    return df

df = timestamp_to_features(df)

# 4.b Randomly assign 2 unique apps for the validation and 2 for the testing

In [1]:
# hyperparameters for picking the apps for validation and test sets
low_appearance_apps_percantage_threshold = 5 # 5%
apps_for_validation = 2
apps_for_test = 2

def select_apps_for_validation_and_test(grouped_stats, apps_for_validation, apps_for_test):

    # Select apps that have a low appearance percentage
    low_representation_apps = grouped_stats[grouped_stats["percentage"]<low_appearance_apps_percantage_threshold]

    # Make a list of the selected apps
    low_representation_apps = low_representation_apps.index.tolist()

    # Shuffle list:
    random.shuffle(low_representation_apps)

    # Select random apps for validation and test sets
    validation_apps = low_representation_apps[:apps_for_validation]
    test_apps = low_representation_apps[apps_for_validation: apps_for_validation + apps_for_test]

    return validation_apps, test_apps

validation_apps, test_apps = select_apps_for_validation_and_test(grouped_stats, apps_for_validation, apps_for_test)

# print the selected apps
print("Validation apps: ", validation_apps)
print("Test apps: ", test_apps)

NameError: name 'grouped_stats' is not defined

# 4.c Split data for training, validationa and testing

In [ ]:

def prepare_data_for_training(df,columns_to_use,validation_apps,test_apps):
    
    # Split the dataset into 80% training and 20% temporary sets
    X_train_temp, X_test = train_test_split(df, test_size=0.2, random_state=42)

    # Further split the temporary set into 70% training and 10% validation sets
    X_train, X_val = train_test_split(X_train_temp, test_size=0.125, random_state=42)

    # move the validation and test apps to the validation and test sets
    # Filter records associated with validation apps from the training set
    train_data_validation_apps = X_train[X_train['bundleId'].isin(validation_apps)]
    X_train = X_train[~X_train['bundleId'].isin(validation_apps)]
    X_val = pd.concat([X_val, train_data_validation_apps])

    # Filter records associated with test apps from the training set
    train_data_test_apps = X_train[X_train['bundleId'].isin(test_apps)]
    X_train = X_train[~X_train['bundleId'].isin(test_apps)]
    X_test = pd.concat([X_test, train_data_test_apps])

    # Shuffling the set to ensure randomness
    X_train = X_train.sample(frac=1, random_state=42)
    X_val = X_val.sample(frac=1, random_state=42)
    X_test = X_test.sample(frac=1, random_state=42)

    # Extract the target 
    y_train = X_train.pop('sentPrice')
    y_val = X_val.pop('sentPrice')
    y_test = X_test.pop('sentPrice')

    # Calculate the proportion of each set
    train_proportion = len(X_train) / len(df)
    val_proportion = len(X_val) / len(df)
    test_proportion = len(X_test) / len(df)

    print("Proportion of training data:", round(train_proportion,4)*100)
    print("Proportion of validation data:", round(val_proportion,4)*100)
    print("Proportion of test data:", round(test_proportion,4)*100)

    # select only the columns_to_use for the training, validation and testing 
    X_train = X_train[columns_to_use]
    X_val = X_val[columns_to_use]
    X_test = X_test[columns_to_use]

    return X_train, X_val, X_test, y_train, y_val, y_test


# Select specific columns for training
columns_to_use = ['bidFloorPrice', 'year', 'month', 'day', 'hour', 'minute', 'second', 'day_of_week']
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data_for_training(df,columns_to_use,validation_apps, test_apps)

# Train the XGBOOST model without considering the apps' embeddings

In [ ]:
def train_model(X_train, X_val, X_test, y_train, y_val, y_test):
    
    # Init the XGBoost model
    model = xgb.XGBRegressor()

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the training set
    y_pred_train = model.predict(X_train)

    # Evaluate the model on training set
    training_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Training Root Mean Squared Error:", training_rmse)

    # Make predictions on the validation set
    y_pred_val = model.predict(X_val)

    # Evaluate the model on validation set
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    print("Validation Root Mean Squared Error:", val_rmse)

    # Make predictions on the testing set
    y_pred_test = model.predict(X_test)

    # Evaluate the model on test set
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    print("Test Root Mean Squared Error:", test_rmse)

    return model, training_rmse, val_rmse, test_rmse

model, training_rmse, val_rmse, test_rmse = train_model(X_train, X_val, X_test, y_train, y_val, y_test)

# 4.d - Clustering the apps' embedding

In [ ]:

def cluster_apps_by_similarity(cos_sim_df, apps_to_exclude):
    threshold_for_similarity = 0.7
    
    # Filter rows and columns based on the apps to exclude
    cos_sim_df = cos_sim_df.drop(columns=apps_to_exclude, index=apps_to_exclude)

    clusters = {}  # Dict to store clusters
    cluster_num = 0
    checked_apps = []
    for current_app, row in cos_sim_df.iterrows():
        if current_app in checked_apps:
            continue
        
        max_similarity_app = row.idxmax()
        max_similarity_value = row.max()

        if max_similarity_value >= threshold_for_similarity:
            # Check if the app with the highest similarity is already in a cluster
            assigned_cluster = None
            for c_num, cluster_data in clusters.items():
                if max_similarity_app in cluster_data['apps']:
                    assigned_cluster = c_num
                    break
            
            # Add the current app to the cluster
            if assigned_cluster is not None:
                clusters[assigned_cluster]['apps'].append(current_app)
                checked_apps.append(current_app)
            
            # Create a new cluster
            if assigned_cluster is None:
                cluster_num += 1
                clusters[cluster_num] = {'apps': [current_app, max_similarity_app]}
                checked_apps.extend([current_app, max_similarity_app])
                

    for c_num, cluster_data in clusters.items():
        apps_in_cluster = cluster_data['apps']
        
        # Get embeddings for apps in the cluster from the embeddings_dict
        cluster_embeddings = [embeddings_dict[app] for app in apps_in_cluster]
        
        # Calculate the average embedding for the cluster
        cluster_data['average_embedding'] = np.mean(cluster_embeddings, axis=0)
    return clusters

apps_to_exclude = validation_apps + test_apps
clusters = cluster_apps_by_similarity(cos_sim_df,apps_to_exclude)


# Assign new apps to the existing

In [ ]:

def assign_apps_to_clusters(apps_to_exclude, clusters, embeddings_dict):
    # Select the best cluster for the apps which were assigned to the test and validation sets
    for app in apps_to_exclude:
        # Get embedding for the app
        current_app_embedding = embeddings_dict[app] 
        # Calculate cosine similarity with average embeddings of each cluster
        similarities = {}
        for cluster_num, cluster_data in clusters.items():
            cluster_embedding = cluster_data['average_embedding']
            similarity = cosine_similarity([current_app_embedding], [cluster_embedding])[0][0]
            similarities[cluster_num] = similarity

        # Assign the app to the cluster with the highest cosine similarity
        max_similarity_cluster = max(similarities, key=similarities.get)
        clusters[max_similarity_cluster]['apps'].append(app)
    
    return clusters

clusters = assign_apps_to_clusters(apps_to_exclude, clusters, embeddings_dict)

# Create one hot encoding for each cluster

In [ ]:
def create_one_hot_encoding(df, clusters):
    
    # iterate over the clusters and create one-hot encoding for the apps
    for cluster_num, cluster_data in clusters.items():
        cluster_apps = cluster_data['apps'] # get the apps in the cluster
        cluster_name = f'cluster_{cluster_num}' # create the cluster name
        df[cluster_name] = 0 # initialize the column with zeros
        df.loc[df['bundleId'].isin(cluster_apps), cluster_name] = 1 # assign 1 to the apps in the cluster

        # return also a list of cluster_name
        cluster_names = [f'cluster_{cluster_num}' for cluster_num in clusters.keys()]
    
    return df,cluster_names

df,cluster_names = create_one_hot_encoding(df, clusters)

# Train a model based on the clusters
Let's repeat the train, test, validation process. This time adding the clusters one hot encoding features 


In [ ]:
# split the data 
columns_to_use = columns_to_use.extend(cluster_names)
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data_for_training(df,columns_to_use,validation_apps,test_apps)

# train the model
model, training_rmse, val_rmse, test_rmse = train_model(X_train, X_val, X_test, y_train, y_val, y_test)

(7) Conclusions
It was found that ...
* It's worth examining additional models except XGBOOST
* Maybe the threshold of clustering the embeding wasn't high enough
* Maybe I should use a different method which is not rely on embedings